In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import sys
import os

sys.path.append(os.path.abspath(".."))

### ChatPromptTemplate

- `ChatPromptTemplate` 는 대화방식의 챗봇을 사용할때는 `PromptTemplate` 보다 조금 더 좋은 성능으로 사용할 수 있습니다.
- AI 와 Human 이 대화를 주고 받는 형식일때 사용합니다.
- `PromptTemplate` 는 지시사항일때 사용합니다.
  
- 메시지는 튜플(tuple)형식으로 구성하며, (role, message) 형태로 작성합니다. 리스트로 생성할 수 있습니다.

##### role

- `system`: 시스템 설정 메세지이며, 전역 설정과 관련한 프롬프트입니다.
- `human`: 사용자 입력 메세지 프롬프트입니다.
- `ai`: AI 의 답변에 관련한 프롬프트입니다.

In [4]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template(
    "{player} 은(는) 어느 팀 소속의 선수인가요?"
)
chat_prompt.format(player="손흥민")

'Human: 손흥민 는 어느 팀 소속의 선수인가요?'

In [6]:
chat_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 친절한 AI 어시드턴트입니다. 당신의 이름은 모비딕이고 직업은 {job} 입니다.",
        ),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    job="pro gammer", user_input="당신의 직업은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시드턴트입니다. 당신의 이름은 모비딕이고 직업은 pro gammer 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 직업은 무엇입니까?', additional_kwargs={}, response_metadata={})]

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

llm.invoke(messages).content

'제 직업은 프로 게이머입니다. 게임을 하면서 수입을 올리는 일을 하고 있어요. 어떤 게임이든 도와드릴 수 있습니다!'

체인을 생성하는 예시입니다.

In [9]:
chain = chat_template | llm
chain.invoke({"job": "pro gammer", "user_input": "당신의 직업은 무엇입니까?"}).content

'제 직업은 프로 게이머입니다. 게임을 하면서 수입을 올리는 일을 하고 있어요. 어떤 게임을 좋아하시나요? 함께 이야기 나누면 즐거울 것 같아요.'

### MessagesPlaceHolder

- 아직 확정된 메세지가 아니지만 나중에 언젠가는 채워질 메시지를 PlaceHolder 로 잡아둡니다.
- 챗봇에서 대화기록을 할 때 사용합니다. 대화기록은 대화중에 확정이 되지 않은 정보이기에 PlaceHolder 를 활용하여 기록합니다.

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("history"),
        ("human", "{question}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['history', 'question'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchai

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage


# LLM 모델 초기화 (OpenAI의 ChatGPT를 사용하는 예시)
llm = ChatOpenAI(model_name="gpt-4o-mini")

# 대화 기록 생성
history = [
    HumanMessage(content="Hello, how are you?"),
    AIMessage(content="I'm doing well, thank you! How can I assist you today?"),
]

# 새로운 질문
question = "딥러닝 4대천황이라고 불리는 사람들은 누구야?"

chain = prompt | llm

chain.invoke({"history": history, "question": question})

AIMessage(content='딥러닝 4대 천황이라고 불리는 사람들은 주로 다음 네 명을 지칭합니다:\n\n1. **제프리 힌튼 (Geoffrey Hinton)** - 딥러닝의 아버지로 불리며, 신경망과 딥러닝의 발전에 큰 기여를 한 인물입니다. 특히 역전파 알고리즘과 딥러닝의 기초 개념들을 발전시켰습니다.\n\n2. **얀 르쿤 (Yann LeCun)** - 합성곱 신경망(CNN)의 발전에 기여한 연구자로, 이미지 인식 분야에서 많은 성과를 올렸습니다. 현재 페이스북 AI 연구소의 수장입니다.\n\n3. **앤드류 응 (Andrew Ng)** - 머신러닝과 딥러닝 분야의 교육 및 연구에 기여한 인물로, 구글 브레인 프로젝트의 공동 창립자이자, 스탠퍼드 대학교 교수입니다.\n\n4. **요슈아 벤지오 (Yoshua Bengio)** - 딥러닝의 이론적 기초를 확립하고, 다양한 신경망 아키텍처를 연구한 인물입니다. 몬트리올 대학교 교수이며, 딥러닝 연구의 선두주자 중 한 명입니다.\n\n이들은 모두 딥러닝 분야에서 중요한 연구와 혁신을 이끌어온 인물들로, 현대 인공지능의 발전에 지대한 영향을 미쳤습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 326, 'prompt_tokens': 62, 'total_tokens': 388, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'f